In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("../data/dataset_clean.csv")

In [10]:
df

,Date,Ticker,Open,High,Low,Close,Volume
0,2005-01-03,MMM,37.516825,38.078417,37.462037,37.621841,3817632.0
1,2005-01-04,MMM,37.621848,37.904925,37.288544,37.315941,4358942.0
2,2005-01-05,MMM,37.302237,37.416381,36.859360,36.859360,3462779.0
3,2005-01-06,MMM,36.927856,37.621852,36.900460,37.192669,3605342.0
4,2005-01-07,MMM,37.210918,37.804464,37.096774,37.576176,3938428.0
...,...,...,...,...,...,...,...
1967898,2024-12-24,ZBH,105.798322,106.432319,105.273287,106.293633,458600.0
1967899,2024-12-26,ZBH,105.996447,107.105945,105.610106,106.244102,1277300.0
1967900,2024-12-27,ZBH,105.451602,106.620535,105.263389,105.867668,743400.0
1967901,2024-12-30,ZBH,105.291816,105.589667,103.901823,104.646461,1532000.0
